In [26]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
# Load the Excel file
file_path = "/Users/esha/Documents/Prescriptive/Final Project/CRE_Data_SpringA2025.xlsx"
xls = pd.ExcelFile(file_path)


In [3]:
# Load the Financials and Equity sheets
financials_df = pd.read_excel(xls, sheet_name="Financials")
equity_df = pd.read_excel(xls, sheet_name="Equity")

In [4]:
# Clean column names
financials_df.columns = financials_df.columns.str.strip()
equity_df.columns = equity_df.columns.str.strip()

In [20]:
# Define weighted objective function to return a scalar value
def weighted_objective(weights):
    return -np.sum(weights[0] * tradeoff_analysis["avg_noi"].mean() +
                   weights[1] * tradeoff_analysis["avg_equity_growth"].mean() -
                   weights[2] * tradeoff_analysis["noi_volatility"].mean() -
                   weights[3] * tradeoff_analysis["vacancy_volatility"].mean() -
                   weights[4] * tradeoff_analysis["op_exp_ratio_volatility"].mean())


In [21]:
# Initial weight values for optimization
init_weights = [0.4, 0.3, 0.1, 0.1, 0.1]

In [22]:
# Constraint: weights must sum to 1
cons = ({'type': 'eq', 'fun': lambda w: sum(w) - 1})

In [23]:
# Bounds: Each weight must be between 0 and 1
bounds = [(0, 1)] * 5


In [27]:
# Run optimization
optimal_weights = minimize(weighted_objective, init_weights, bounds=bounds, constraints=cons)

In [28]:
# Apply optimized weights
tradeoff_analysis["Weighted Score"] = (
    optimal_weights.x[0] * tradeoff_analysis["avg_noi"] +
    optimal_weights.x[1] * tradeoff_analysis["avg_equity_growth"] -
    optimal_weights.x[2] * tradeoff_analysis["noi_volatility"] -
    optimal_weights.x[3] * tradeoff_analysis["vacancy_volatility"] -
    optimal_weights.x[4] * tradeoff_analysis["op_exp_ratio_volatility"]
)

In [36]:
# Compute Initial Equity Investment Per City
equity_summary = equity_df.groupby("City").agg(
    avg_initial_equity=("Initial Equity Investment", "mean")
).reset_index()

In [37]:
# Merge Initial Equity Data with tradeoff_analysis
tradeoff_analysis = tradeoff_analysis.merge(equity_summary, on="City", how="left")

In [38]:
# Compute Return on Investment (ROI)
tradeoff_analysis["ROI"] = (
    tradeoff_analysis["avg_noi"] + (tradeoff_analysis["avg_equity_growth"] * tradeoff_analysis["avg_initial_equity"])
) / tradeoff_analysis["avg_initial_equity"]

In [41]:
# Compute Total Risk Exposure as a weighted sum of different volatilities
tradeoff_analysis["Total Risk Exposure"] = (
    tradeoff_analysis["noi_volatility"] * 0.5 + 
    tradeoff_analysis["vacancy_volatility"] * 0.3 + 
    tradeoff_analysis["op_exp_ratio_volatility"] * 0.2
)

In [42]:
# Compute MOO Score (Risk-Adjusted ROI)
tradeoff_analysis["MOO Score"] = (
    tradeoff_analysis["ROI"] / tradeoff_analysis["Total Risk Exposure"]
)

In [43]:
# Normalize the MOO Scores (Scaling between 0-1 for comparison)
tradeoff_analysis["MOO Score"] = (tradeoff_analysis["MOO Score"] - tradeoff_analysis["MOO Score"].min()) / (
    tradeoff_analysis["MOO Score"].max() - tradeoff_analysis["MOO Score"].min()
)

In [44]:
print(tradeoff_analysis[["City", "ROI", "Total Risk Exposure", "MOO Score"]])

          City       ROI  Total Risk Exposure  MOO Score
0     Fairview  6.135842        192563.677724   0.850494
1     New Hope  7.998054        240738.179077   1.000000
2  Springfield  4.995328        206995.228909   0.000000


In [45]:
best_market = tradeoff_analysis.loc[tradeoff_analysis["MOO Score"].idxmax(), "City"]
print(f"Best investment market based on MOO: {best_market}")

Best investment market based on MOO: New Hope


In [29]:
# Identify the best city based on Multi-Objective Optimization
best_market = tradeoff_analysis.loc[tradeoff_analysis["Weighted Score"].idxmax(), "City"]


In [30]:
print(f"Best investment market based on MOO: {best_market}")

Best investment market based on MOO: New Hope
